In [19]:
###IMPORTANT ####
 # Actor until 1976
 # Actor in a Leading Role since 1977
 # Same for Actress
    
### Stock the data better than doing a request every try # Done

### PROBLEME : COMMENT GERER LES 2 WINNERS, existe plusieurs fois # Done

### PROBLEM : for DIRECTION, names and titles are swapped # Done


### PROBLEME : Incohérence des noms de catégories sur plusieurs année : best motion picture -> best picture
# Special effect -> Visual effects etc

### Les informations récoltées ne sont pas les mêmes ex : foreign film c'est le pays qui est récolté, comment faire ?

In [1]:
### OOP   ### DONE

import time
import requests
from scrapy import Selector
from tqdm import tqdm
import pandas as pd

class Oscar_Scraper:
    
    list_names = []
    list_year = []
    list_category = []
    list_films = []
    list_results = []
    
    def __init__(self, from_time, to_time):
        self.links = {
            'pages':[]
        }
        
        self.categories_interest = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role',
                  'Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        self.categories_individuals = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role']
        self.categories_films = ['Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        #lists of different names for same categories
        self.categories_dictionary = {
                    "Actor": ['Actor', 'Actor in a Leading Role'],
                    "Actress": ['Actress', 'Actress in a Leading Role'],
                    "Picture": ['Outstanding Picture', 'Outstanding Production','Best Motion Picture', 'Best Picture'],
                    "VE" : ['Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)','Visual Effects']
                    }
        
        
        self.from_time = from_time
        self.time_period = range(from_time, to_time + 1)
        self.data = {
            'category':[],
            'film':[],
            'name':[],
            'gender':[],
            'result':[],
            'birthdate':[]
        }
        
        

    def getHTML(self):
        
        for i in self.time_period:
            #time.sleep(10) # Crawlerdelay
            url = "https://www.oscars.org/oscars/ceremonies/{}".format(i)
            response_oscars = requests.get(url)
            sel = Selector(text = response_oscars.text)   # Scrapy chose over BeautifulSoup for selector CSS
            self.links['pages'].append(sel)

    def getNominees(self, years, i, category, number_nominees):

        for number in range(number_nominees):   
            # NAME
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
            nominee_name = ''.join(self.links['pages'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            # FILM
            path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
            nominee_film = ''.join(self.links['pages'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
            
            self.list_year.append(self.from_time + years)         
            self.list_category.append(category)
            self.list_results.append("Nominee")

    def getWinners(self, years, i, category, number_winners):

        if number_winners == 1: # 1 Winner
        # NAME
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['pages'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)
            
        
            # FIlM
            path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
            nominee_film = ''.join(self.links['pages'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
           
            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Winner")


        if number_winners == 2: # 2 Winners
            # NAME 1st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['pages'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # NAME 2st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['pages'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # FILM
            path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
            path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

            nominee_film = ''.join(self.links['pages'][years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)

            winner2_film = ''.join(self.links['pages'][years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
            winner2_film = winner2_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)        



    def getDATA(self):
            
        for years in tqdm(range(len(self.links['pages']))):

            categories = self.links['pages'][years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
            
            for i in range(len(categories)):
                if categories[i] in self.categories_individuals:

                    # Need the number of winners (might be ties) and nominated          
                    path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
                    number_of_film_nominated = int(''.join(self.links['pages'][years].css(path_nominated).extract()).split("views-row-")[1])


                    path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
                    trial = ''.join(self.links['pages'][years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
                    if len(trial) == 2:
                        number_winners = 1
                    elif len(trial) == 1:
                        number_winners = 2
                    else:
                        print("problem")

                    ## WINNERS                 
                    self.getWinners(years, i, categories[i], number_winners)     
                    ## NOMINEES
                    self.getNominees(years, i, categories[i], number_of_film_nominated )   
                    
                    
        self.data = {
            'year': self.list_year,
            'category': self.list_category,
            'film': self.list_films,
            'name': self.list_names,
            #'gender':[],
            'result':self.list_results
            #,
            #'birthdate':[]
        }            


    def getINDIVIDUALS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        individuals = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_individuals]]
        return(individuals)
    
    def getFILMS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        films = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_interest[5:]]]
        films.columns = ('film', 'Studio/Creator(s)', 'result')
        return(films)
    
     #creating big dataframe with all data
    def getAllCategories(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        return(df)
    
    #in progress
    """ #creating CSV files based on scraped data. Possible naming of the columns for different datasets
    def create_named_CSV(self,dataframe,categories,columns_name):
        if len(column_name) > 2:
          raise Exception('Items exceeds the maximum allowed length of 2')
        category = []
        if type(categories) == 'str':
        category.append(categories)
        for x in self.categories_dictionary:
            for y in range(len(self.categories_dictionary[x])):
                if category == self.categories_dictionary[x][y]:
                    if x not in names:
                        names.append(x)
        for x in names:
            create_empty_CSV(x)
        df_final.columns = columns_name
             films.to_csv(i + '.csv', sep=' ', mode='a')"""
        
    #creating empty csv names after argument of the method   
    def create_empty_CSV(*stringcsv_names):
        for i in stringcsv_names:
            with open(i + '.csv', 'w') as f:
                pass
    

In [2]:
test = Oscar_Scraper(1990,2000)
test.getHTML()
test.getDATA()



100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 14.22it/s]


In [87]:
### OOP   ### ROTTENTOMATOES

import time
import requests
from scrapy import Selector
from tqdm import tqdm
import pandas as pd
import re  # text processing
import numpy as np
import json

class Oscar_Scraper:
    
    list_names = []
    list_year = []
    list_category = []
    list_films = []
    list_results = [] 
    list_ratings_imdb = []
    list_ratings_rottentomatoes = []
    list_metacritics = []
    list_boxoffice = []
    
    
    def __init__(self, from_time, to_time):
        self.links = {
            'oscars':[],
            'rotten':[],
        }
        
        self.categories_interest = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role',
                  'Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        self.categories_individuals = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role']
        self.categories_films = ['Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        #lists of different names for same categories
        self.categories_dictionary = {
                    "Actor": ['Actor', 'Actor in a Leading Role'],
                    "Actress": ['Actress', 'Actress in a Leading Role'],
                    "Picture": ['Outstanding Picture', 'Outstanding Production','Best Motion Picture', 'Best Picture'],
                    "VE" : ['Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)','Visual Effects']
                    }
        
        
        self.from_time = from_time
        self.time_period = range(from_time, to_time + 1)
        self.data = {
            'category':[],
            'film':[],
            'year':[],
            'name':[],
            'gender':[],
            'result':[],
            'birthdate':[],
            'imbd_ratings':[],
            'rotten_ratings':[],
            'metacritics_ratings':[],
            'box office':[]
            
        }
        
        

    def getHTML(self):
        
        for i in self.time_period:
            #time.sleep(10) # Crawlerdelay
            url = "https://www.oscars.org/oscars/ceremonies/{}".format(i)
            response_oscars = requests.get(url)
            sel = Selector(text = response_oscars.text)   # Scrapy chose over BeautifulSoup for selector CSS
            self.links['oscars'].append(sel)

    def getNominees(self, years, i, category, number_nominees, number_winners):

        for number in range(number_nominees):   
            if number_winners == 1:
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
                nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
            if number_winners == 2:
                 # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 +number)
                nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
            
            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            # FILM
            if number_winners == 1:
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
                nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
            if number_winners == 2:
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 +number)
                nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
            
            self.list_year.append(self.from_time + years)         
            self.list_category.append(category)
            self.list_results.append("Nominee")

    def getWinners(self, years, i, category, number_winners):

        if number_winners == 1: # 1 Winner
        # NAME
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)
            
        
            # FIlM
            path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
            nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
           
            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Winner")


        if number_winners == 2: # 2 Winners
            # NAME 1st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # NAME 2st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # FILM
            path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
            path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

            nominee_film = ''.join(self.links['oscars'][years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)

            winner2_film = ''.join(self.links['oscars'][years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
            winner2_film = winner2_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(winner2_film)
            else:
                self.list_films.append(winner2_film)        



    def getDATA(self):
            
        for years in tqdm(range(len(self.links['oscars']))):

            categories = self.links['oscars'][years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
            
            for i in range(len(categories)):
                if categories[i] in self.categories_individuals:

                    # Need the number of winners (might be ties) and nominated          
                    path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
                    number_of_film_nominated = int(''.join(self.links['oscars'][years].css(path_nominated).extract()).split("views-row-")[1])

                    path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
                    trial = ''.join(self.links['oscars'][years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
                    if len(trial) == 2:
                        number_winners = 1
                    elif len(trial) == 1:
                        number_winners = 2
                    else:
                        print("problem")


                    ## WINNERS                 
                    self.getWinners(years, i, categories[i], number_winners)     
                    ## NOMINEES
                    self.getNominees(years, i, categories[i], number_of_film_nominated, number_winners)   
                    
                    
        self.data = {
            'year': self.list_year,
            'category': self.list_category,
            'film': self.list_films,
            'name': self.list_names,
            #'gender':[],
            'result':self.list_results
            #,
            #'birthdate':[]
        }            

        ###Individual corrections
        for i in range(len(self.data["film"])):
            if self.data["film"][i] == "Good Fellas":
                self.data["film"][i] = "Goodfellas"
            if self.data["film"][i] == "Life Is Beautiful":
                self.data["film"][i] = "La vita è bella"
            if self.data["film"][i] == "Pelle the Conqueror":
                self.data["film"][i] = "Pelle Erobreren"
            if self.data["film"][i] == "Roland Joffé":
                self.data["film"][i] = "The Killing Fields"
                self.data["name"][i] = "Roland Joffé"
            if self.data["film"][i] == "Mulholland Drive":
                self.data["film"][i] = "Mulholland Dr."
            if self.data["film"][i] == "Precious: Based on the Novel 'Push' by Sapphire":
                self.data["film"][i] = "Precious"
            if self.data["film"][i] == "Demián Bichir":
                self.data["film"][i] = "A Better Life"
                self.data["name"][i] = "Demián Bichir"
            if self.data["film"][i] == "Two Days, One Night":
                self.data["film"][i] = "Deux jours, une nuit"
            if self.data["film"][i] == "Five Fingers" and self.data["year"][i] == 1953:
                self.data["film"][i] = "5 Fingers"
            if self.data["film"][i] == "Adventures of Robinson Crusoe":
                self.data["film"][i] = "Robinson Crusoe"
            if self.data["film"][i] == "Henry V" and self.data["year"][i] == 1947:
                self.data["film"][i] = "The Chronicle History of King Henry the Fifth with His Battell Fought at Agincourt in France"    
            if self.data["film"][i] == "All That Money Can Buy":
                self.data["film"][i] = "The Devil and Daniel Webster"
            if self.data["film"][i] == "Two Women":
                self.data["film"][i] = "La ciociara"
            if self.data["film"][i] == "Divorce--Italian Style":
                self.data["film"][i] = "Divorzio all'italiana"    
            if self.data["film"][i] == "Federico Fellini's 8-1/2":
                self.data["film"][i] = "8½"
            if self.data["film"][i] == "Zorba the Greek":
                self.data["film"][i] = "Aléxis Zorbás"  
            if self.data["film"][i] == "The Emigrants":
                self.data["film"][i] = "Utvandrarna"
            if self.data["film"][i] == "Last Tango in Paris":
                self.data["film"][i] = "Ultimo tango a Parigi"
            if self.data["film"][i] == "Cries and Whispers":
                self.data["film"][i] = "Viskningar och rop"   
            if self.data["film"][i] == "A Special Day":
                self.data["film"][i] = "Una giornata particolare"   
            
    
            
                
    def getINDIVIDUALS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        individuals = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_individuals]]
        return(individuals)
    
    def getFILMS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        films = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_interest[5:]]]
        films.columns = ('film', 'Studio/Creator(s)', 'result')
        return(films)
    
     #creating big dataframe with all data
    def getAllCategories(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        return(df)
    
    #in progress
    """ #creating CSV files based on scraped data. Possible naming of the columns for different datasets
    def create_named_CSV(self,dataframe,categories,columns_name):
        if len(column_name) > 2:
          raise Exception('Items exceeds the maximum allowed length of 2')
        category = []
        if type(categories) == 'str':
        category.append(categories)
        for x in self.categories_dictionary:
            for y in range(len(self.categories_dictionary[x])):
                if category == self.categories_dictionary[x][y]:
                    if x not in names:
                        names.append(x)
        for x in names:
            create_empty_CSV(x)
        df_final.columns = columns_name
             films.to_csv(i + '.csv', sep=' ', mode='a')"""
        
    #creating empty csv names after argument of the method   
    def create_empty_CSV(*stringcsv_names):
        for i in stringcsv_names:
            with open(i + '.csv', 'w') as f:
                pass
            
######################################################### 
    
    def getAPI_omdb(self):
        key = "59d5666b"
        url_api = "https://www.omdbapi.com/?t={}&y={}&apikey={}"
        for film_number in tqdm(range(len(self.data['film']))):
            film = self.data['film']
            title_standard = film[film_number].upper().replace(" ","+").replace(".","").replace("&","%26").replace("ONCE+UPON+A+TIMEIN+HOLLYWOOD", "ONCE+UPON+A+TIME...+IN+HOLLYWOOD")
            year_standard = self.data['year'][film_number]-1
            response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
            if response_api["Response"] != "False":
                for ratings in range(len(response_api["Ratings"])):
                    
                    if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                        self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])
                        
                    if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                        self.list_metacritics.append(response_api["Ratings"][ratings]["Value"]) 
                    
                if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                    self.list_ratings_rottentomatoes.append("N/A")
                if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                    self.list_metacritics.append("N/A")
                        
                self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))            
                self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))
            
                
            else:
                year_standard = self.data['year'][film_number]-2
                response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                if response_api["Response"] != "False":
                    for ratings in range(len(response_api["Ratings"])):
                        if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                            self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])
                    
                        if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                            self.list_metacritics.append(response_api["Ratings"][ratings]["Value"])
                      
                    if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                        self.list_ratings_rottentomatoes.append("N/A")
                    if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                        self.list_metacritics.append("N/A")                                        
                    self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))          
                    self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))
                    
               
                else:
                    year_standard = self.data['year'][film_number]-3
                    response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                    if response_api["Response"] != "False":
                        for ratings in range(len(response_api["Ratings"])):
                            if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                                self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])

                            if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                                self.list_metacritics.append(response_api["Ratings"][ratings]["Value"])

                        if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                            self.list_ratings_rottentomatoes.append("N/A")
                        if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                            self.list_metacritics.append("N/A")                                        
                        self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))          
                        self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))
                    else:
                        year_standard = self.data['year'][film_number]
                        response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                        if response_api["Response"] != "False":
                            for ratings in range(len(response_api["Ratings"])):
                                if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                                    self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])

                                if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                                    self.list_metacritics.append(response_api["Ratings"][ratings]["Value"])

                            if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                                self.list_ratings_rottentomatoes.append("N/A")
                            if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                                self.list_metacritics.append("N/A")                                        
                            self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))          
                            self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))
                        else:
                            title_standard = title_standard.replace("%26","and")
                            year_standard = self.data['year'][film_number]-1
                            response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                            if response_api["Response"] != "False":
                                for ratings in range(len(response_api["Ratings"])):
                                    if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                                        self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])

                                    if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                                        self.list_metacritics.append(response_api["Ratings"][ratings]["Value"])

                                if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                                    self.list_ratings_rottentomatoes.append("N/A")
                                if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                                    self.list_metacritics.append("N/A")                                        
                                self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))          
                                self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))

                            else:
                                year_standard = self.data['year'][film_number]-2
                                response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                                if response_api["Response"] != "False":
                                    for ratings in range(len(response_api["Ratings"])):
                                        if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                                            self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])

                                        if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                                            self.list_metacritics.append(response_api["Ratings"][ratings]["Value"])

                                    if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                                        self.list_ratings_rottentomatoes.append("N/A")
                                    if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                                        self.list_metacritics.append("N/A")                                        
                                    self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))          
                                    self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))
                                else: 
                                    year_standard = self.data['year'][film_number]-3
                                    response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                                    if response_api["Response"] != "False":
                                        for ratings in range(len(response_api["Ratings"])):
                                            if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                                                self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])

                                            if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                                                self.list_metacritics.append(response_api["Ratings"][ratings]["Value"])

                                        if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                                            self.list_ratings_rottentomatoes.append("N/A")
                                        if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
                                            self.list_metacritics.append("N/A")                                        
                                        self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))          
                                        self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))

                                    else:
                                        print("Error",film[film_number],title_standard,year_standard)
            
        # Standardization
        
        for i in tqdm(range(len(self.data['film']))): 
            if self.list_boxoffice[i] == "N/A":
                self.list_boxoffice[i] = np.nan
            else:
                self.list_boxoffice[i] = float(self.list_boxoffice[i].replace("$","").replace(",",""))

            if self.list_ratings_imdb[i] == "N/A":
                self.list_ratings_imdb[i] = np.nan
            else:
                self.list_ratings_imdb[i] = float(self.list_ratings_imdb[i])

            if self.list_ratings_rottentomatoes[i] == "N/A":
                self.list_ratings_rottentomatoes[i] = np.nan
            else:
                self.list_ratings_rottentomatoes[i] = float(self.list_ratings_rottentomatoes[i].replace("%",""))

            if self.list_metacritics[i] == "N/A":
                self.list_metacritics[i] = np.nan
            else:
                self.list_metacritics[i] = float(self.list_metacritics[i].replace("/100",""))
    
        self.data['imbd_ratings'] = self.list_ratings_imdb,
        self.data['rotten_ratings'] = self.list_ratings_rottentomatoes,
        self.data['metacritics_ratings'] = self.list_metacritics,
        self.data['box office'] = self.list_boxoffice
                                         

In [88]:
test = Oscar_Scraper(1930,2021)
test.getHTML()
test.getDATA()


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:07<00:00, 12.31it/s]


In [89]:
test.getAPI_omdb()
##PROBLEME : PAS DE BOX OFFICE et METACRITICS

# Problème avec Once Upon a Time in Hollywood (reportage, pas le film), ce n'est pas le seul, mais oublié lequel c'était l'autre

100%|██████████████████████████████████████████████████████████████████████████| 1372/1372 [00:00<00:00, 228320.31it/s]


In [90]:

        

print(test.list_boxoffice)


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1526000.0, nan, nan, nan, nan, nan, nan, nan, nan, 1526000.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 8066000.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1500000.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 2000000.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 200852579.0, 624643.0, nan, 144738.0, 200852579.0, nan, nan, nan, nan, 200852579.0, nan, 144738.0, nan, 624643.0, 404524.0, nan, nan, nan, nan, nan, nan, nan, 404524.0, nan, nan, nan, nan, 404524.0, nan, 16361885.0, nan, nan, nan, 1585634.0, nan, nan, na

In [47]:
for i in range(len(test.data))

['Charly',
 'The Heart Is a Lonely Hunter',
 'The Fixer',
 'Oliver!',
 'The Lion in Winter',
 'The Lion in Winter',
 'Funny Girl',
 'The Subject Was Roses',
 'Isadora',
 'Rachel, Rachel',
 'Oliver!',
 'The Battle of Algiers',
 'The Lion in Winter',
 'Romeo and Juliet',
 '2001: A Space Odyssey']

In [45]:

url_rotten = "https://www.rottentomatoes.com/m/lglfo"
print(url_rotten)
response_rotten = requests.get(url_rotten)
if response_rotten.status_code == requests.codes.ok:
    print("tkt bg")
else:
    print("ça fonfonctionne")
    print(response_rotten.status_code)
    
        for i in tqdm(self.data['film']):
            #time.sleep(10) # Crawlerdelay
            standardization = i.lower().replace("&", "and").replace("the","").replace(".","").replace("'", "").replace(",", " ").replace(" ", "_")
            url_rotten = "https://www.rottentomatoes.com/m/{}".format(standardization)
            response_rotten = requests.get(url_rotten)
            if response_rotten.status_code == requests.codes.ok:
                sel = Selector(text = response_rotten.text)   # Scrapy chose over BeautifulSoup for selector CSS
                self.links['rotten'].append(sel)
            else:
                print(url_rotten)

https://www.rottentomatoes.com/m/lglfo
ça fonfonctionne
404


In [84]:
key = "59d5666b"
url_api = "https://www.omdbapi.com/?t=THE+UNITED+STATES+VS+BILLIE+HOLIDAY&y=2021&apikey=59d5666b"

test = requests.get(url_api)


In [85]:
import json
json.loads(test.text)


{'Title': 'The United States vs. Billie Holiday',
 'Year': '2021',
 'Rated': 'R',
 'Released': '26 Feb 2021',
 'Runtime': '126 min',
 'Genre': 'Biography, Drama, Music',
 'Director': 'Lee Daniels',
 'Writer': 'Suzan-Lori Parks, Johann Hari',
 'Actors': 'Andra Day, Trevante Rhodes, Garrett Hedlund',
 'Plot': 'Follows Holiday during her career as she is targeted by the Federal Department of Narcotics with an undercover sting operation led by black Federal Agent Jimmy Fletcher, with whom she had a tumultuous affair.',
 'Language': 'English, Italian, German',
 'Country': 'United States',
 'Awards': 'Nominated for 1 Oscar. 7 wins & 20 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BN2FjZDU0MGMtZDkxMi00M2VmLTkyNDQtNTFjYjIzYzMxMjgwXkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '55%'},
  {'Source': 'Metacritic', 'Value': '52/100'}],
 'Metascore': '52',
 'imdbRat